# Push/Pull to AWS
* Training DF files
* then push other data

In [ ]:
import pandas as pd
import boto3
import numpy as np
import os
from os import listdir
from os.path import isfile, join
from tqdm import tqdm_notebook
import warnings
from pathlib import Path

warnings.filterwarnings("ignore")
#set path directory

#load access key
#HOME = os.path.expanduser('~')
HOME = os.getcwd()
HOME = os.chdir('..')
HOME = os.getcwd()
# HOME = os.getcwd()
KEYPATH = "utils/AWSaccessKeys.csv"
ACCESS = pd.read_csv(f"{HOME}/{KEYPATH}")

#start session
SESSION = boto3.Session(
    aws_access_key_id=ACCESS['Access key ID'][0],
    aws_secret_access_key=ACCESS['Secret access key'][0],
)
S3 = SESSION.resource('s3')
#AWS BUCKET information
BUCKET_NAME = 'national-snow-model'
BUCKET = S3.Bucket(BUCKET_NAME)


In [ ]:
HOME

## Training DF regional files

In [ ]:
#Upload nearest-snotel dictionaries
regionlist = ['SouthernRockies', 'Southwest', 'Northwest']
output_res = 300

for region in regionlist:
    print(region)
    head_folder = f"SWEMLv2.0/data/TrainingDFs/{region}/300M_Resolution"
    head_folder_dir = f"{HOME}/{head_folder}"

    #Get files only
    files = [f for f in listdir(head_folder_dir) if isfile(join(head_folder_dir, f))]
    # for file in tqdm_notebook(files):
    #     S3.meta.client.upload_file(Filename= f"{head_folder_dir}{file}", Bucket=BUCKET_NAME, Key=f"{head_folder}{file}")

    #get list of directories to download
    dirs = [ f.path for f in os.scandir(head_folder_dir) if f.is_dir() ]
    for dir in dirs:
        print(f"Folder name: {dir.split('300M_Resolution/')[-1]}")
        awsfolderpath = f"SWEMLv2.0{dir.split('SWEMLv2.0')[-1]}"
        headfolderpath = f"{HOME}/{awsfolderpath}"
        files = [f for f in listdir(headfolderpath) if isfile(join(headfolderpath, f))]
        # for file in tqdm_notebook(files):
        #      S3.meta.client.upload_file(Filename= f"{headfolderpath}/{file}", Bucket=BUCKET_NAME, Key=f"{awsfolderpath}/{file}")
        #     # print(f"{headfolderpath}/{file}")

        # # #Get subfolders
        subdirs = [ f.path for f in os.scandir(headfolderpath) if f.is_dir() ]
        for dir in subdirs:
            print(f"Folder name: {dir.split('300M_Resolution/')[-1]}")
            awsfolderpath2 = f"SWEMLv2.0{dir.split('SWEMLv2.0')[-1]}"
            headfolderpath2 = f"{HOME}/{awsfolderpath2}"
            files = [f for f in listdir(headfolderpath2) if isfile(join(headfolderpath2, f))]
            for file in tqdm_notebook(files):
                S3.meta.client.upload_file(Filename= f"{headfolderpath2}/{file}", Bucket=BUCKET_NAME, Key=f"{awsfolderpath2}/{file}")



        




## Get files

In [ ]:
#Upload nearest-snotel dictionaries
regionlist = ['Northwest','SouthernRockies', 'Southwest' ]
output_res = 500

#Get data
for region in regionlist:
    print(region)
    path = f"SWEMLv2.0/data/TrainingDFs/{region}/{output_res}M_Resolution"

    #Make directory if it does not exist
    if not os.path.exists(f"{HOME}/{path}"):
        print("Path not present, making")
        os.makedirs(f"{HOME}/{path}", exist_ok=True)
    
    #Identify the potential files to download
    files = [objects.key for objects in BUCKET.objects.filter(Prefix=path)]
    #print(files)
    
    for file in tqdm_notebook(files):
        #Make directory if it does not exist
        if not os.path.exists(f"{HOME}/{path}"):
            print("Path not present, making")
        os.makedirs( os.path.dirname(f"{HOME}/{file}"), exist_ok=True)
       
        #check to see if the file is there
        if os.path.exists(f"{HOME}/{file}") == False:
            print('Downloading file')
            S3.Bucket(BUCKET_NAME).download_file(file, f"{HOME}/{file}")

        else:
            print('File already present...')


In [ ]:
os.path.dirname(f"{HOME}/Test/{file}")

In [ ]:
HOME

In [ ]:
#Upload nearest-snotel dictionaries
regionlist = ['Northwest', 'Northwest'] #'SouthernRockies', 'Southwest' ]
output_res = 500

#Get data
for region in regionlist:
    print(region)

In [ ]:
# Just want to list all prefixes to see file structure 
client = SESSION.client('s3')

response = client.list_objects_v2(
    Bucket=BUCKET_NAME,
    Delimiter='/'
)

prefixes = response.get('CommonPrefixes', [])
for prefix in prefixes:
    print(prefix['Prefix'])

In [ ]:
# for obj in BUCKET.objects.filter(Prefix='SWEMLv2.0/Predictions'):
    # print(obj.key)


In [ ]:
# Push model predictions 

WY_list = [2013,2024]
# WY_list = [2014]
thresh = 10
res = 1000
df = 'NLDASDaymet_Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs'
folder = f'{HOME}/Predictions/{df}/{res}M_Resolution/{thresh}_fSCA_Thresh'

for year in WY_list:
    filepath = f'{folder}/{year}/GeoTiffs'
    files = [file for file in os.listdir(filepath) if file.endswith('tif')]
    for file in files:
        localpath = f'{filepath}/{file}'
        if os.path.isfile(localpath) == True:
            # print(localpath)
            key = f'SWEMLv2.0/Predictions/{res}M_Resolution/{thresh}_fSCA_Thresh/{year}/{file}'
            BUCKET.upload_file(
                Filename=localpath,
                Key=key
                )

            # print(key)